# Perceptrón aplicado a iris

**Lectura del corpus:** $\;$ comprobamos también que las matrices de datos y etiquetas tienen las filas y columnas que toca

In [ ]:
import numpy as np; from sklearn.datasets import fetch_olivetti_faces
iris = fetch_olivetti_faces(); X = iris.data.astype(np.float16);
y = iris.target.astype(np.uint).reshape(-1, 1);
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data
(400, 4096) (400, 1) 
 [[0.30981445 0.36767578 0.41723633 ... 0.16113281 0.15698242 0.        ]
 [0.45458984 0.47119141 0.51220703 ... 0.15283203 0.15283203 0.        ]
 [0.31811523 0.40087891 0.49169922 ... 0.14880371 0.15283203 0.        ]
 [0.19836426 0.19421387 0.19421387 ... 0.75195312 0.73974609 0.        ]
 [0.5        0.54541016 0.58251953 ... 0.17358398 0.17358398 0.        ]]


**Partición del corpus:** $\;$ Creamos un split de iris con un $20\%$ de datos para test y el resto para entrenamiento (training), barajando previamente los datos de acuerdo con una semilla dada para la generación de números aleatorios. Aquí, como en todo código que incluya aleatoriedad (que requiera generar números aleatorios), conviene fijar dicha semilla para poder reproducir experimentos con exactitud.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

(320, 4096) (80, 4096)


**Implementación de Perceptrón:** $\;$ devuelve pesos en notación homogénea, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ también el número de errores e iteraciones ejecutadas

In [ ]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

**Aprendizaje de un clasificador (lineal) con Perceptrón:** $\;$ Perceptrón minimiza el número de errores de entrenamiento (con margen)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{Y}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [ ]:
W, E, k = perceptron(X_train, y_train)
print("Número de iteraciones ejecutadas: ", k)
print("Número de errores de entrenamiento: ", E)
print("Vectores de pesos de las clases (en columnas y en notación homogénea):\n", W);

Número de iteraciones ejecutadas:  42
Número de errores de entrenamiento:  0
Vectores de pesos de las clases (en columnas y en notación homogénea):
 [[-587.         -581.         -581.         ... -590.
  -557.         -585.        ]
 [-247.08929443 -233.21960449 -244.16125488 ... -266.64141846
  -244.67816162 -239.23931885]
 [-265.13061523 -251.18511963 -267.29406738 ... -281.07702637
  -270.37103271 -256.94555664]
 ...
 [-211.28652954 -203.89273071 -199.72198486 ... -196.6746521
  -201.48944092 -207.01318359]
 [-205.98419189 -202.98080444 -198.18414307 ... -194.52960205
  -195.22445679 -199.60604858]
 [-206.72164917 -203.47958374 -198.56820679 ... -189.42913818
  -192.849823   -194.80505371]]


**Cálculo de la tasa de error en test:**
El siguiente conjunto de instrucciones calcula la tasa de error para un Perceptron entrenado con b=0.1 y a=1.0 (factor de aprendizaje o 'a') y K=200 iteraciones

In [ ]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Tasa de error en test: {err_test:.1%}")

Tasa de error en test: 7.5%


In [ ]:
err_test * len(X_test)

6.0

**Ajuste del margen:** $\;$ el siguiente bucle es un experimento que ejecuta 5 veces el algoritmo del Perceptron con 5 valores diferentes de $b$ y valor por defecto $α$=1.0 y K=1000 iteraciones, mostrando el error de entrenamiento para cada valor de $b$.

In [ ]:
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)
    print(b, E, k)

0.0 0 48
0.01 0 58
0.1 0 42
10 0 45
100 0 72


**Interpretación de resultados:** $\;$ los datos de entrenamiento no parecen linealmente separables; no está claro que un margen mayor que cero pueda mejorar resultados, sobre todo porque solo tenemos $30$ muestras de test; con margen nulo ya hemos visto que se obtiene un error (en test) del $16.7\%$
-

In [ ]:
min_error = float('inf')
best_combination = None
print('# a b K E k Ete')
for a in (0.1,1.0,10,100,1000,10000):
  for b in (0.0,0.01,0.1,1.0,100,1000):
    for K in (200,500,800,1000):
      W, E, k = perceptron(X_train, y_train, b, a, K)
      X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
      y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
      err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
      print('%8.2f %8.2f %6d %6d %6d %8.3f' %(a, b, K, E,k,err_test))
      if err_test < min_error:
                min_error = err_test
                best_combination = (a, b, k)

  print('#------- ------ ---- ---- ---- ----')
print('Mejor combinación con menor error:', best_combination)
print('Menor error encontrado:', min_error)

# a b K E k Ete
    0.10     0.00    200      0     60    0.125
    0.10     0.00    500      0     60    0.125
    0.10     0.00    800      0     60    0.125
    0.10     0.00   1000      0     60    0.125
    0.10     0.01    200      0     42    0.075
    0.10     0.01    500      0     42    0.075
    0.10     0.01    800      0     42    0.075
    0.10     0.01   1000      0     42    0.075
    0.10     0.10    200      0     79    0.113
    0.10     0.10    500      0     79    0.113
    0.10     0.10    800      0     79    0.113
    0.10     0.10   1000      0     79    0.113
    0.10     1.00    200      0     45    0.150
    0.10     1.00    500      0     45    0.150
    0.10     1.00    800      0     45    0.150
    0.10     1.00   1000      0     45    0.150
    0.10   100.00    200      0     96    0.062
    0.10   100.00    500      0     96    0.062
    0.10   100.00    800      0     96    0.062
    0.10   100.00   1000      0     96    0.062
    0.10  1000.00    200

**Ejercicio para realizar en clase:**
---


Escribe el código python necesario para mostrar resultados del error de entrenamiento (E), número de iteraciones empleadas (k) y error de test (err_test) para combinaciones de diferentes valores de:
---

# *   a: (factor de aprendizaje: utiliza valores 0.1, 1.0, 10, 100, 1000, 10000)
# *   b: (margen: utiliza valores 0.0, 0.01, 0.1, 1.0, 100, 1000)
# *   K: (iteraciones: utiliza valores 200, 500, 800 y 1000)


Utiliza la siguientes instrucciones para mostrar la cabecera:
---

`print('#      a        b      K      E      k      Ete');`
---
`print('#-------   ------   ----   ----   ----     ----');`
---

y la siguiente instrucción para mostrar los resultados:

`print('%8.2f %8.2f %6d %6d %6d %8.3f' %(a, b, K, E,k,err_test))`
---





Después de completar la ejecución del programa, podemos extraer conclusiones importantes de los datos obtenidos. En primer lugar, se observa una tendencia clara: a medida que el factor de aprendizaje aumenta, el error de prueba también lo hace, siendo los errores más pequeños notables en valores de aprendizaje de 0.1. Además, se nota que un mayor número de iteraciones generalmente se asocia con un menor error, con algunas excepciones donde a menor número de iteraciones se presenta un menor error de prueba, como se observa en el caso particular de a = 10000.

A partir de estas observaciones, no es sorprendente concluir que la mejor configuración para este caso de estudio sería un factor de aprendizaje de 0.1, un margen de 1000.0, k igual a 412. Sin embargo, es importante destacar que en ciertos casos particulares, como a = 10000, la elección óptima puede diferir de esta norma. Estos resultados sugieren la importancia de considerar cuidadosamente el conjunto de parámetros y sus interacciones para lograr un rendimiento óptimo en el modelo.